In [1]:
import prior

dataset = prior.load_dataset("procthor-10k")
dataset

[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 12180.46it/s]


DatasetDict(
    train=Dataset(
    dataset=procthor-dataset,
    size=10000,
    split=train
),
    val=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=val
),
    test=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=test
)
)

In [2]:
from ai2thor.controller import Controller


house = dataset["train"][3]
controller = Controller(scene=house, snapToGrid=False, rotateStepDegrees=30, renderInstanceSegmentation=True)
event = controller.step("Pass")
spawn = event.metadata["agent"]["position"]

/home/juyuanli/miniconda3/envs/nav_assistant/lib/python3.11/site-packages/ai2thor/platform.py:154: UserWarning: could not connect to X Display: 5, Can't connect to display ":5": b'Authorization required, but no authorization protocol specified\n'
  warnings.warn(


# PCA Step

In [3]:
from rl import PPO, CLIPNovelty, ClipEnv, ActorCritic
from models import FrozenResNetEncoder, SlidingWindowTransformerActor, SlidingWindowTransformerCritic
from cons import FEAT_DIM, NUM_ACTIONS

ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetEncoder(project_to_out_dim=False)
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS) # Not important
critic = SlidingWindowTransformerCritic(FEAT_DIM) # Not important
clip_novelty = CLIPNovelty()
clip_env = ClipEnv(clip_novelty)
clip_actor_critic = ActorCritic(encoder, actor, critic)

In [4]:
from rl import inference, teleport
import torch

def get_distribution(ppo, obs_seq, actions_seq, actor_critic):
    dist = torch.distributions.Categorical(probs=torch.tensor([0.5, 0.25, 0.25]))
    return dist

all_obs = []
for i in range(12):
    event = teleport(controller)
    init_pos = event.metadata["agent"]["position"]
    obs = inference(get_distribution=get_distribution, controller=controller, ppo=ppo, init_position=init_pos, env=clip_env, actor_critic=clip_actor_critic, plot=False, n=32)
    all_obs.append(torch.stack(obs, dim=0))

all_obs_tensor = torch.cat(all_obs, dim=0)
embedding = encoder(all_obs_tensor.unsqueeze(0)).squeeze(0)

from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components=FEAT_DIM)
pca.fit(embedding.detach().cpu().numpy())
W = pca.components_.astype(np.float32)

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

...

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

# Training

In [5]:
from rl import PPO, SegmentationNovelty, ClipEnv, ActorCritic, train
from models import FrozenResNetPCAEncoder, SlidingWindowTransformerActor, SlidingWindowTransformerCritic
from cons import FEAT_DIM, NUM_ACTIONS, DEVICE

ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetPCAEncoder(FEAT_DIM, torch.from_numpy(W), device=DEVICE)
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS) # Not important
critic = SlidingWindowTransformerCritic(FEAT_DIM) # Not important
clip_novelty = SegmentationNovelty()
clip_env = ClipEnv(clip_novelty)
clip_actor_critic = ActorCritic(encoder, actor, critic)

In [ ]:
train(controller, "segment_novelty", ppo, clip_env, clip_actor_critic, 50)

wandb: Currently logged in as: viriyadhika-putra (viriyadhika1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[PPO] Epoch 0: Loss=4.8189, Policy=0.0026, Value=9.7154
[PPO] Epoch 10: Loss=4.7138, Policy=0.0102, Value=9.5053
[PPO] Epoch 20: Loss=4.4677, Policy=-0.0011, Value=9.0334
[PPO] Epoch 30: Loss=3.9189, Policy=-0.0104, Value=7.9508
Approx KL Learned: 0.027324900031089783
[✅] Actor-Critic checkpoint saved to data/segment_novelty.pt
Update 1/50 — steps: 4096
[PPO] Epoch 0: Loss=6.3773, Policy=0.0018, Value=12.8441
[PPO] Epoch 10: Loss=5.7486, Policy=0.0011, Value=11.5931
[PPO] Epoch 20: Loss=5.0750, Policy=-0.0076, Value=10.2616
[PPO] Epoch 30: Loss=4.5990, Policy=-0.0074, Value=9.3099
Approx KL Learned: 0.016432728618383408
[✅] Actor-Critic checkpoint saved to data/segment_novelty.pt
Update 2/50 — steps: 4096
[PPO] Epoch 0: Loss=8.7508, Policy=0.0031, Value=17.5852
[PPO] Epoch 10: Loss=7.5847, Policy=-0.0014, Value=15.2680
[PPO] Epoch 20: Loss=6.9987, Policy=-0.0028, Value=14.1023
[PPO] Epoch 30: Loss=6.2788, Policy=-0.0059, Value=12.6683
Approx KL Learned: 0.011562875472009182
[✅] Actor-C

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[✅] Actor-Critic checkpoint saved to data/segment_novelty_49.pt
[✅] Actor-Critic checkpoint saved to data/segment_novelty.pt
Update 50/50 — steps: 4096


episode_reward,▁▁▅▄▅▃▁▃▄▃▅▄▄▅█▄▃▄▅▅▆▅▆▆▅▅▄▆▄▆▇█▇▆▆█▆▅▇█
reward,▁▁▂▂▁▂▁▁▁▂▂▂▂▁▁▂▂▁▂▂▂▂▂█▂▂▂▂▂▂▂▁▁▂▂▁▂▁▂▂
episode_reward,0.25331
reward,0.10937


(<rl.RolloutBuffer at 0x7f911824f450>, [], [])

In [ ]:
train(controller, "segment_novelty_next_50", ppo, clip_env, clip_actor_critic, 50)

[PPO] Epoch 0: Loss=5.4822, Policy=0.0083, Value=11.0465
[PPO] Epoch 10: Loss=3.2865, Policy=-0.0039, Value=6.6817
[PPO] Epoch 20: Loss=2.0924, Policy=-0.0251, Value=4.3334
[PPO] Epoch 30: Loss=1.6953, Policy=-0.0401, Value=3.5690
Approx KL Learned: 0.0412585623562336
[✅] Actor-Critic checkpoint saved to data/segment_novelty_next_50.pt
Update 1/50 — steps: 4096
[PPO] Epoch 0: Loss=4.2952, Policy=0.0074, Value=8.6708
[PPO] Epoch 10: Loss=3.0680, Policy=-0.0162, Value=6.2636
[PPO] Epoch 20: Loss=1.9428, Policy=-0.0390, Value=4.0574
[PPO] Epoch 30: Loss=1.4921, Policy=-0.0516, Value=3.1814
Approx KL Learned: 0.03610769659280777
[✅] Actor-Critic checkpoint saved to data/segment_novelty_next_50.pt
Update 2/50 — steps: 4096
[PPO] Epoch 0: Loss=6.5999, Policy=0.0008, Value=13.2935
[PPO] Epoch 10: Loss=5.5923, Policy=-0.0160, Value=11.3135
[PPO] Epoch 20: Loss=3.9983, Policy=-0.0283, Value=8.1493
[PPO] Epoch 30: Loss=2.6181, Policy=-0.0362, Value=5.4049
Approx KL Learned: 0.03836238384246826
[

ConnectionResetError: Connection lost

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f9134241790>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f9117e737d0, execution_count=7 error_before_exec=None error_in_exec=Connection lost info=<ExecutionInfo object at 7f930783ad90, raw_cell="train(controller, "segment_novelty_next_50", ppo, .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a225975616e6c69616e675f6e65775f736572766572227d/home/juyuanli/Viri/NavAssistant/RL/segment.ipynb#W6sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

# Minimum segment novelty

In [5]:
from rl import PPO, SegmentationSizeNovelty, ClipEnv, ActorCritic, train
from models import FrozenResNetPCAEncoder, SlidingWindowTransformerActor, SlidingWindowTransformerCritic
from cons import FEAT_DIM, NUM_ACTIONS, DEVICE

ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetPCAEncoder(FEAT_DIM, torch.from_numpy(W), device=DEVICE)
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS) # Not important
critic = SlidingWindowTransformerCritic(FEAT_DIM) # Not important
clip_novelty = SegmentationSizeNovelty()
clip_env = ClipEnv(clip_novelty)
clip_actor_critic = ActorCritic(encoder, actor, critic)

In [6]:
train(controller, "segment_size_novelty", ppo, clip_env, clip_actor_critic, 50)

wandb: Currently logged in as: viriyadhika-putra (viriyadhika1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[PPO] Epoch 0: Loss=7.2872, Policy=0.0014, Value=14.6785
[PPO] Epoch 10: Loss=6.9987, Policy=0.0089, Value=14.0823
[PPO] Epoch 20: Loss=6.3092, Policy=0.0042, Value=12.7137
[PPO] Epoch 30: Loss=5.0994, Policy=-0.0113, Value=10.3284
Approx KL Learned: 0.050216734409332275
[✅] Actor-Critic checkpoint saved to data/segment_size_novelty.pt
Update 1/50 — steps: 4096
[PPO] Epoch 0: Loss=5.8679, Policy=-0.0039, Value=11.8499
[PPO] Epoch 10: Loss=4.9143, Policy=-0.0045, Value=9.9440
[PPO] Epoch 20: Loss=4.1507, Policy=-0.0081, Value=8.4244
[PPO] Epoch 30: Loss=3.5283, Policy=-0.0074, Value=7.1779
Approx KL Learned: 0.025707658380270004
[✅] Actor-Critic checkpoint saved to data/segment_size_novelty.pt
Update 2/50 — steps: 4096
[PPO] Epoch 0: Loss=4.8481, Policy=0.0025, Value=9.7996
[PPO] Epoch 10: Loss=3.9072, Policy=-0.0061, Value=7.9351
[PPO] Epoch 20: Loss=3.0570, Policy=-0.0128, Value=6.2478
[PPO] Epoch 30: Loss=2.4570, Policy=-0.0161, Value=5.0542
Approx KL Learned: 0.019599663093686104
[✅

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[✅] Actor-Critic checkpoint saved to data/segment_size_novelty_49.pt
[✅] Actor-Critic checkpoint saved to data/segment_size_novelty.pt
Update 50/50 — steps: 4096


episode_reward,▁▃▂▂▂▂▂▃▄▃▄▂▃▄▄▅▂▅▂▂▃▅▃▄▃▃▅▅▅▅▅▆▆▂▆▅▇▇█▇
reward,▆▇▇▆▅▆▇▆▆▆▇▆▇▆▇▆▆▆▅▇▆▇▇▆█▇▇▇▇▇▇▂▁▄▇█▇█▄▇
episode_reward,0.36892
reward,0.72265


(<rl.RolloutBuffer at 0x7fb6e9c87790>, [], [])

In [5]:
controller.stop()

In [ ]:
train(controller, "segment_size_novelty_next_50", ppo, clip_env, clip_actor_critic, 50)

[PPO] Epoch 0: Loss=10.5678, Policy=0.0078, Value=21.2007
[PPO] Epoch 10: Loss=7.9733, Policy=-0.0054, Value=16.0363
[PPO] Epoch 20: Loss=6.2645, Policy=-0.0252, Value=12.6597
[PPO] Epoch 30: Loss=3.9755, Policy=-0.0366, Value=8.1030
Approx KL Learned: 0.04182656109333038
[✅] Actor-Critic checkpoint saved to data/segment_size_novelty_next_50.pt
Update 1/50 — steps: 4096
[PPO] Epoch 0: Loss=8.9436, Policy=0.0092, Value=17.9481
[PPO] Epoch 10: Loss=5.5964, Policy=-0.0088, Value=11.2923
[PPO] Epoch 20: Loss=3.7946, Policy=-0.0269, Value=7.7247
[PPO] Epoch 30: Loss=1.9432, Policy=-0.0387, Value=4.0432
Approx KL Learned: 0.04228238761425018
[✅] Actor-Critic checkpoint saved to data/segment_size_novelty_next_50.pt
Update 2/50 — steps: 4096
[PPO] Epoch 0: Loss=6.9282, Policy=0.0091, Value=13.9233
[PPO] Epoch 10: Loss=3.6268, Policy=-0.0082, Value=7.3555
[PPO] Epoch 20: Loss=2.1298, Policy=-0.0292, Value=4.4012
[PPO] Epoch 30: Loss=1.3748, Policy=-0.0495, Value=2.9334
Approx KL Learned: 0.0551

# New Env

In [6]:
from ai2thor.controller import Controller


house = dataset["train"][8084]
controller = Controller(scene=house, snapToGrid=False, rotateStepDegrees=30, renderInstanceSegmentation=True)
event = controller.step("Pass")
spawn = event.metadata["agent"]["position"]

/home/juyuanli/miniconda3/envs/nav_assistant/lib/python3.11/site-packages/ai2thor/platform.py:154: UserWarning: could not connect to X Display: 5, Can't connect to display ":5": b'Authorization required, but no authorization protocol specified\n'
  warnings.warn(


In [7]:
from rl import PPO, CLIPNovelty, ClipEnv, ActorCritic
from models import FrozenResNetEncoder, SlidingWindowTransformerActor, SlidingWindowTransformerCritic
from cons import FEAT_DIM, NUM_ACTIONS

ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetEncoder(project_to_out_dim=False)
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS) # Not important
critic = SlidingWindowTransformerCritic(FEAT_DIM) # Not important
clip_novelty = CLIPNovelty()
clip_env = ClipEnv(clip_novelty)
clip_actor_critic = ActorCritic(encoder, actor, critic)

In [8]:
from rl import inference, teleport
import torch

def get_distribution(ppo, obs_seq, actions_seq, actor_critic):
    dist = torch.distributions.Categorical(probs=torch.tensor([0.5, 0.25, 0.25]))
    return dist

all_obs = []
for i in range(12):
    event = teleport(controller)
    init_pos = event.metadata["agent"]["position"]
    obs = inference(get_distribution=get_distribution, controller=controller, ppo=ppo, init_position=init_pos, env=clip_env, actor_critic=clip_actor_critic, plot=False, n=32)
    all_obs.append(torch.stack(obs, dim=0))

all_obs_tensor = torch.cat(all_obs, dim=0)
embedding = encoder(all_obs_tensor.unsqueeze(0)).squeeze(0)

from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components=FEAT_DIM)
pca.fit(embedding.detach().cpu().numpy())
W = pca.components_.astype(np.float32)

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

...

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

In [9]:
from rl import PPO, SegmentationNovelty, ClipEnv, ActorCritic, train
from models import FrozenResNetPCAEncoder, SlidingWindowTransformerActor, SlidingWindowTransformerCritic
from cons import FEAT_DIM, NUM_ACTIONS, DEVICE

ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetPCAEncoder(FEAT_DIM, torch.from_numpy(W), device=DEVICE)
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS) # Not important
critic = SlidingWindowTransformerCritic(FEAT_DIM) # Not important
clip_novelty = SegmentationNovelty()
clip_env = ClipEnv(clip_novelty)
clip_actor_critic = ActorCritic(encoder, actor, critic)

In [10]:
train(controller, "segment_novelty_8084", ppo, clip_env, clip_actor_critic, 50)

wandb: Currently logged in as: viriyadhika-putra (viriyadhika1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[PPO] Epoch 0: Loss=4.4926, Policy=0.0040, Value=9.0719
[PPO] Epoch 10: Loss=4.2983, Policy=0.0041, Value=8.6807
[PPO] Epoch 20: Loss=3.9243, Policy=-0.0036, Value=7.9487
[PPO] Epoch 30: Loss=3.4815, Policy=0.0021, Value=7.0552
Approx KL Learned: 0.021547865122556686
[✅] Actor-Critic checkpoint saved to data/segment_novelty_8084.pt
Update 1/50 — steps: 4096
[PPO] Epoch 0: Loss=6.4106, Policy=-0.0018, Value=12.9216
[PPO] Epoch 10: Loss=5.7790, Policy=-0.0019, Value=11.6606
[PPO] Epoch 20: Loss=5.2364, Policy=-0.0040, Value=10.5802
[PPO] Epoch 30: Loss=4.6872, Policy=-0.0062, Value=9.4848
Approx KL Learned: 0.008490501902997494
[✅] Actor-Critic checkpoint saved to data/segment_novelty_8084.pt
Update 2/50 — steps: 4096
[PPO] Epoch 0: Loss=4.5438, Policy=-0.0003, Value=9.1888
[PPO] Epoch 10: Loss=3.8211, Policy=-0.0047, Value=7.7517
[PPO] Epoch 20: Loss=3.2532, Policy=-0.0085, Value=6.6232
[PPO] Epoch 30: Loss=2.8641, Policy=-0.0100, Value=5.8486
Approx KL Learned: 0.004321023356169462
[✅]

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[✅] Actor-Critic checkpoint saved to data/segment_novelty_8084_49.pt
[✅] Actor-Critic checkpoint saved to data/segment_novelty_8084.pt
Update 50/50 — steps: 4096


episode_reward,▅▃▅▂▃▄▄▅▃▃▄▆▄▅▅▄▄▁▃▅▄▆▆▆▆▇█▄▅▆▆▆▅▆▇▃▇▆▆█
reward,▆▆▇▆▅▆▆▆▁▆▆▆▆▆▆▅▇▆▆▇▆▇▂▃▇▆▇▇█▇██▇▇▆▇▇▂██
episode_reward,0.30408
reward,0.49729


(<rl.RolloutBuffer at 0x7fa4a5167cd0>, [], [])